# Hands-On 1 — Deep Shielding and Variance Reduction with Geant4

**47th School of Computing – Latin America 2026**  
Hands-On Session 1

**Lecturer:** Jaime Romero-Barrientos   
**Contact:** jaime.romero@cern.ch   
Researcher — Chilean Nuclear Energy Commission (CCHEN)  
Researcher — Millennium Institute for Subatomic Physics at the High-Energy Frontier (SAPHIR)

<div style="display:flex; align-items:center; gap:32px;">
  <img src="https://cernbox.cern.ch/remote.php/dav/public-files/3XD5887poIjRXe6/logo_CCHEN.jpg" width="180"/>
  <img src="https://cernbox.cern.ch/remote.php/dav/public-files/foYapCY948XIY91/Logo-Saphir.png" width="200"/>
</div>

---
## Overview <a name="overview"></a>

This hands-on introduces Monte Carlo simulation techniques for **deep shielding problems** using **Geant4**, with emphasis on:

- Deep-penetration, rare-event observables
- Statistical uncertainty and convergence
- Variance reduction techniques
- Performance assessment using the Figure of Merit (FOM)

You will run, analyze, and compare **analog** and **variance-reduced** simulations for a fixed physical problem, learning how statistical efficiency can be  improved **without modifying the underlying physics**.

---
## Computing environment <a name="computing-environment"></a>

This hands-on is executed using:

- **Geant4 version:** 11.3.2  
- **Physics list:** FTFP_BERT_HP  
- **Computing platform:** SWAN (CERN Jupyter service)  
- **Storage:** CERNBox  
- **Software stack:** LCG 108 (LHC Computing Grid)

All simulations are run inside the **SWAN environment**, using CPU resources and a controlled software stack, to ensure a **common and reproducible setup** for all participants.

---
## Physical problem: deep neutron shielding <a name="physical-problem-deep-neutron-shielding"></a>

In this hands-on we study **neutron transport through a thick concrete shield**, a prototypical **deep shielding problem** in radiation transport.

In such problems, the quantities of interest are dominated by **rare particle histories** that survive many interactions and reach deep regions of the shield.  
This makes the estimation of physical observables **statistically challenging** and an ideal testbed for Monte Carlo methods.

---

## Geometry and particle transport <a name="geometry-and-particle-transport"></a>

The simulated geometry consists of:

- A **10 MeV neutron source** located at `z = -90.0005 cm`, emitting particles along the `+z` axis.
- A **cylindrical concrete shield** aligned with the beam axis, with **radius = 100 cm** and **length = 90 cm**.
- A longitudinal subdivision into **20 consecutive scoring cells** (`cell_00` … `cell_19`):

  - `cell_00`: upstream region (closest to the source), **vacuum (G4_Galactic)**  
  - `cell_01` … `cell_18`: intermediate **concrete** slices  
  - `cell_19`: downstream region (deep-penetration ROI), **vacuum (G4_Galactic)**

As neutrons propagate through the concrete, they:
- Scatter
- Lose energy
- Are absorbed

As a result:
- The particle population decreases rapidly with depth
- Only a very small fraction of histories reaches the deepest cells

This rapid attenuation makes estimators in deep regions (such as `cell_19`)
**noisy and slow to converge**.

---

## Geometry visualization (illustrative)

The following figures illustrate the simulated geometry and particle transport.

> ⚠️ **Important note**  
> The SWAN computing environment does not provide a graphical visualization interface for Geant4.
> Therefore, these images are shown **for conceptual understanding only**.
> All simulations in this hands-on are executed in **batch mode**.

<figure>
  <img src="https://cernbox.cern.ch/remote.php/dav/public-files/6kS1LLi0qi19UcE/geometry_1_HO1.png" style="width:60%">
  <figcaption>Overall geometry of the deep shielding problem.</figcaption>
</figure>

<figure>
  <img src="https://cernbox.cern.ch/remote.php/dav/public-files/VesUtjTPNa24rOS/geometry_2_HO1.png" style="width:60%">
  <figcaption>Longitudinal subdivision of the concrete shield into scoring cells.</figcaption>
</figure>

---

## Scoring and physical meaning of the output <a name="scoring-and-physical-meaning-of-the-output"></a>

### Scoring quantities

For each cell (`cell_00` … `cell_19`), the simulation reports several estimators.
The most relevant ones for this exercise are:

- **Track-length (SL)**  
  The total particle track length accumulated in the cell, averaged per primary event.

- **Weighted track-length (SLW)**  
  The track-length multiplied by the particle statistical weight.  
  This is the **unbiased estimator** used when variance reduction techniques are enabled.

In this hands-on, the first runs are **fully analog**:
- All particle weights are equal to 1
- Therefore, `SL` and `SLW` are numerically identical

---

### Interpreting the per-cell table

For each cell, the output table includes:

- `Tr.Entering`  
  Number of track entries into the cell

- `Population`  
  A measure of how much particle history lives in that cell

- `Av.Tr.WGT`  
  Average particle weight (equal to 1 in analog runs)

- `SL` and `Sigma(SL)`  
  Mean track-length estimator and its statistical uncertainty

- `SLW` and `Sigma(SLW)`  
  Weight-corrected estimator and its uncertainty

---

## Region of interest and goal <a name="region-of-interest-and-goal"></a>

The **region of interest (ROI)** in this hands-on is:

- **`cell_19`**, the deepest cell in the shield

Because very few particles reach this region, its track-length estimator
is dominated by rare events and exhibits large statistical fluctuations.

## Goal of this hands-on

Your objective is to estimate the **track-length estimator in `cell_19`**
with a **relative statistical uncertainty below 10%**:

$$
R = \frac{\sigma}{\bar{x}} < 10\%
$$

To achieve this, you will:
- Run simulations with increasing statistics
- Analyze the convergence of the estimator
- Measure performance using wall-clock time and the Figure of Merit (FOM)

---
## Simulation output <a name="simulation-output"></a>

Each simulation run produces output in two forms:

1. **Screen output** (printed in the notebook cell)
2. **Text files written to disk** (saved automatically by the code)

You must be able to read and interpret both.

---

## Screen output

### Per-cell results

For each scoring cell (`cell_00` … `cell_19`), the code prints a table containing:

- `Tr.Entering`  
- `Population`  
- `Av.Tr.WGT`  
- `SL` and `Sigma(SL)`  
- `SLW` and `Sigma(SLW)`

These quantities are reported **per primary event**.

In the analog runs used in this hands-on:
- All particle weights are equal to 1
- Therefore, `SL` and `SLW` are numerically identical

---

### Region-of-interest summary

At the end of each run, a summary block is printed for the **region of interest (`cell_19`)**, including:

- `Mean(SLW)`
- `StdErr(SLW)`
- `Relative error (%)`
- `Wall-clock time (s)`
- `Figure of Merit (FOM)`

The **Figure of Merit** is defined as:

$$
\text{FOM} = \frac{1}{R^2 \cdot T}
$$

where:
- $R$ is the relative statistical error
- $T$ is the wall-clock time

---

## Files written to disk (`build/out/`)

In addition to the screen output, each run writes text files to disk.

### Per-run cell tables
For each run, a file containing the full per-cell scoring tables is written:

- `b01_cells_mode{mode}_run{runID}_N{Nevt}.tsv`

where:
- `{mode}` indicates the simulation mode (`0` for analog simulations and `-1` for biased ones)
- `{runID}` is the run identifier (usually `0`)
- `{Nevt}` is the number of primary events.

So if you run an analog simulation with N=100,000 primary events, your output file would be:

- `b01_cells_mode0_run0_N100000.tsv`

---

### Cumulative summary file

A cumulative summary file is also **updated**:

- `b01_summary.tsv`

This file collects, for each run:
- Number of events
- Mean estimator value in `cell_19`
- Statistical uncertainty
- Relative error
- Execution time
- FOM

You will use this file later to compare runs and analyze convergence.

## Download and compilation <a name="download-and-compilation"></a>

We will now download the simulation code from GitHub and compile it inside SWAN.

In [ ]:
#
# Executing this cell will download and compile the exercise
#
from time import time
import os
import shutil
from pathlib import Path
import subprocess

t1 = time()

# --- Workspace setup ---
%cd
%mkdir -p CSC26/VR
%cd CSC26/VR

base_dir = Path.cwd()
repo_dir = base_dir / "hands_on_1"

print("Obtaining source code from GitHub...")

# --- Robust clone (safe to re-run the cell) ---
if repo_dir.exists():
    print("Repository already exists. Removing it to ensure a clean setup...")
    shutil.rmtree(repo_dir)

!git --no-pager clone https://github.com/jromero-barrientos/hands_on_1.git

# --- Build directory ---
%mkdir -p hands_on_1/build
%cd hands_on_1/build

print("Running cmake...")
!cmake ..

print("Running make...")
!make -j

# --- Compilation check ---
exe = Path("exampleB01")
if exe.is_file() and os.access(exe, os.X_OK):
    print(f"✅ Compilation successful: '{exe.name}' found.")
else:
    raise RuntimeError(
        "❌ Compilation failed: executable 'exampleB01' not found.\n"
        "Check the output above."
    )

# --- Optional sanity run ---
print("Running a short test run (100 events)...")

mode = -1   # analog mode
N = 100

env = os.environ.copy()
env["B01_MODE"] = str(mode)

# IMPORTANT: use explicit path to the executable
subprocess.run([f"./{exe.name}", str(mode), str(N)], env=env, check=True)

print("✅ Test run completed")

# --- Final info ---
print("Current working directory:", os.getcwd())

t2 = time()
print("Installed exercise in {:.2f} minutes".format((t2 - t1) / 60.0))

## Compilation and sanity check <a name="compilation-and-sanity-check"></a>

If everything worked correctly, you should have observed the following:

- The source code was successfully downloaded from GitHub.
- The project compiled without errors using **Geant4 11.3.2** (LCG release 108 on SWAN).
- An executable called **`exampleB01`** was created inside the `build/` directory.
- A short test run with **100 events** was executed successfully.

During the test run, you should have seen:
- A table printed to screen with one row per cell (`cell_00` … `cell_19`).
- Columns such as:
  - `Tr.Entering`
  - `Population`
  - `Av.Tr.WGT`
  - `SL` and `Sigma(SL)`
  - `SLW` and `Sigma(SLW)`
- A summary at the end showing the **track-length estimator in `cell_19`**, together with its uncertainty and execution time (unless the mean, sigma or time are zero)

👉 **At this stage, do not worry about the numerical values.**  

This run is only meant to verify that:
- The code compiles correctly.
- The simulation runs as expected.
- You are familiar with the structure of the output.

In the next steps, we will start running simulations with larger numbers of particles and analyze the results in detail.

#### Important: You should now be located in `~/CSC26/VR/hands_on_1/build/.`

If you are not sure, run:
- `!pwd`

If needed, go back to the build directory with:
- `%cd ~/CSC26/VR/hands_on_1/build`

In [ ]:
!pwd

## First analog run <a name="first-analog-run"></a>

We now perform the **first analog simulation** with non-trivial statistics.

### Run configuration

- **Run mode:** analog (no variance reduction)
- **Number of events:**  
  
  N_1 = 100,000
  
- **All particle weights:** equal to 1

This run serves as a **baseline**:

- It allows us to understand the structure of the output.
- It shows how particle population and estimators evolve with depth.
- It provides a first estimate of the track-length in the deep cell (`cell_19`).

### What to pay attention to

When examining the output table:

- How do `Population` and `Tr.Entering` change with cell index?
- Compare shallow cells (e.g. `cell_03`) with deep cells (`cell_19`).
- Verify that:
  - `Av.Tr.WGT = 1`
  - `SL ≡ SLW` (as expected for an analog run)

At the end of the output, note the **ROI summary** for `cell_19`:
- Mean(SLW)
- StdErr(SLW)
- Relative error (%)
- Run time
- FOM

You will record these values in the next cell.

In [ ]:
# First analog run: N = 100,000 events

import os, subprocess
from pathlib import Path

base_dir  = Path.home() / "CSC26/VR/hands_on_1"
build_dir = base_dir / "build"
exe = build_dir / "exampleB01"  # estando en build/
mode = -1
N = 100000

env = os.environ.copy()
env["B01_MODE"] = str(mode)

subprocess.run([str(exe), str(mode), str(N)], env=env, check=True)

print("\nRun completed.")
print("Check the screen output and the files written in build/out/")

---

## Interpreting the results of the first analog run <a name="interpreting-the-results-of-the-first-analog-run"></a>

After running the first analog simulation (`N = 100,000`), carefully inspect:

- The **per-cell table** printed on screen.
- The **ROI summary for `cell_19`** printed at the end of the run.

Answer the following questions **in your own words**.

---

### 1. Particle population and depth

Look at the columns **`Tr.Entering`** and **`Population`**.

- How do these quantities change as we move from `cell_00` to `cell_19`?
- What physical processes in a concrete shield explain this behavior?
- Why does the population drop so dramatically in the deepest cells?

---

### 2. Average track weight

Examine the column **`Av.Tr.WGT`**.

- What value does it take in all cells?
- Why is this expected in an **analog** simulation?
- What would it physically mean if this value were different from 1?

---

### 3. Track-length estimators: SL vs SLW

Compare **`SL`** and **`SLW`** for all cells.

- Are they numerically identical?
- Why does this happen in this run?
- Which estimator would remain valid if particle weights were not equal to 1?

---

### 4. Statistical uncertainties and depth

Focus on **`Sigma(SL)`** and **`Sigma(SLW)`**.

- How do the uncertainties change as a function of cell index?
- Compare the relative error in a shallow cell (e.g. `cell_03`) with that in the deepest cell (`cell_19`).
- Why is the relative error in `cell_19` still large despite running 100,000 events?

---

### 5. Region of Interest (cell_19): cost vs precision

Now look at the **ROI summary for `cell_19`**, printed at the end of the run.

- Write down:
  - `Mean(SLW)`
  - `StdErr(SLW)`
  - **Relative error (%)**
  - **Execution time (s)**
  - **FOM (1/s)**

- Is the relative uncertainty below the target **10%**?
- How much computational time was required to reach this precision?

---

### 6. First conclusions

Based on this first run only:

- Is the statistical precision in `cell_19` acceptable for our goal?
- What do you expect would happen to:
  - the uncertainty,
  - the execution time,
if the number of events were increased?

---

## Second analog run: increasing statistics <a name="second_analog_run_increasing_statistics"></a>

The first run with **100,000 events** showed that the statistical uncertainty
in the deepest cell (`cell_19`) is still very large.

To reduce the relative uncertainty, we now increase the number of events
by **one full order of magnitude**.

In the next run you will execute an **analog simulation with**:

- **N = 1,000,000 events**

After this run, you will:
- Repeat the same analysis as before.
- Compare the results with the 100k run.
- Pay special attention to:
  - `cell_03` (shallow region),
  - `cell_19` (deep penetration region),
  - execution time and FOM.

Do **not** change any simulation parameters other than `N`.

In [ ]:
# Second analog run: N = 1,000,000 events

import os, subprocess
from pathlib import Path

base_dir  = Path.home() / "CSC26/VR/hands_on_1"
build_dir = base_dir / "build"
exe = build_dir / "exampleB01"  # estando en build/
mode = -1
N = 1000000

env = os.environ.copy()
env["B01_MODE"] = str(mode)

subprocess.run([str(exe), str(mode), str(N)], env=env, check=True)

print("\nRun completed.")
print("Check the screen output and the files written in build/out/")

## Comparing analog runs: 100k vs 1,000,000 events <a name="comparing_analog_runs"></a>

You have now executed **two analog simulations** of the *same physical problem*:

- Run 1: **N = 100,000**
- Run 2: **N = 1,000,000**

The **only difference** between the two runs is the number of particle histories.

This comparison allows you to study:
- Statistical convergence
- Cost versus precision
- The practical limits of brute-force Monte Carlo

---

### 1. Shallow regions (where statistics are abundant)

Focus on a shallow cell, for example **`cell_03`**, where many particle histories contribute.

Compare between the two runs:
- `SL` and `SLW`
- `Sigma(SL)` and `Sigma(SLW)`
- Their respective relative errors

**Questions:**
- Does the estimator value change noticeably?
- Does the statistical uncertainty decrease when increasing N by a factor 10?
- Is this behavior consistent with what you expect from Monte Carlo statistics?

What does this tell you about estimator behavior in regions with **good statistics**?

---

### 2. Deep penetration region (where statistics are scarce)

Now focus on the **region of interest**, **`cell_19`**.

Compare between the two runs:
- `Mean(SLW)` and `StdErr(SLW)`
- Relative error (%)
- Execution time and FOM

**Questions:**
- Did the precision improve when increasing N by a factor 10?
- By how much did the relative error decrease?
- Are you anywhere near the **10%** target?
- Based on this comparison, does “just increasing N” look like a practical strategy?

---

### Interim conclusion

From these two runs, you should already observe a clear contrast:

- In shallow regions, increasing N behaves exactly as expected.
- In deep regions, convergence is **much slower** and increasingly expensive.

---

➡️ **Next step**

You will now be given the results of a much larger **analog run**
(**10 million events**) executed on the same SWAN environment.

Using that run, you will:
- Quantify how uncertainty scales with N
- Estimate the computational cost required to reach tighter targets
  (e.g. **10%** and **1%** relative uncertainty)

## Importing a precomputed high-statistics analog run (10 million events)

We will now work with the results of an **analog simulation with 10,000,000 events**
that has been **precomputed for you**.

### Important: this is exactly the same simulation

This run is identical to the ones you have already executed:

- Same geometry (20 longitudinal scoring cells)
- Same materials
- Same physics list (**FTFP_BERT_HP**)
- Same scoring quantities
- Same output format
- Same execution environment (SWAN / LCG 108 / Geant4 11.3.2)

✅ **The only difference is the number of events:**
simwv
$$
N = 10{,}000{,}000
$$

---

### Why are we importing this run instead of running it ourselves?

Simply because of **execution time**.

Running 10 million events:
- Takes **several minutes** on SWAN
- Would consume a large fraction of the hands-on session
- Would lead to idle waiting time rather than analysis and discussion

Instead, we provide this run so that you can:
- Inspect realistic high-statistics results
- Use them as a **reference point** for scaling arguments
- Quantify the true cost of brute-force Monte Carlo

👉 When you load the file, **pay close attention to the wall-clock time**
reported for this 10M run and compare it with your previous runs.

**Question (think before answering):**  
Based on that time, would it be realistic for everyone to run 10M events
during this hands-on session?

---

### What we will do next

In the next code cell, we will:

- Load your own analog summary file (e.g. from the 100k or 1M run), if available
- Load the precomputed **10M analog summary and per-cell table**
- Use these results **for analysis only**
  - We will **not modify** your original output files

These data will allow us to:
- Study how uncertainty scales with the number of events
- Estimate the computational cost required to reach specific precision targets
- Decide whether brute-force analog simulation is a viable strategy

In [ ]:
import pandas as pd
from pathlib import Path

# --- Base folders (robust: no $HOME string issues)
base_dir  = Path.home() / "CSC26/VR/hands_on_1"
build_dir = base_dir / "build"
out_dir   = build_dir / "out"
pre_dir   = base_dir / "pcruns"

# --- Files
student_summary  = out_dir / "b01_summary.tsv"
precomp_summary  = pre_dir / "b01_summary_10M.tsv"
precomp_cells10m = pre_dir / "b01_cells_mode-1_run0_N10000000.tsv"

print("=== Paths ===")
print("base_dir        :", base_dir)
print("build_dir       :", build_dir)
print("out_dir         :", out_dir)
print("pre_dir         :", pre_dir)
print("student_summary :", student_summary)
print("precomp_summary :", precomp_summary)
print("precomp_cells10m:", precomp_cells10m)

# --- Load student's summary (if any)
if student_summary.exists():
    df_student = pd.read_csv(student_summary, sep="\t", comment="#")
    print(f"\nLoaded STUDENT summary: {student_summary}  ({len(df_student)} runs)")
else:
    df_student = pd.DataFrame()
    print("\nNo student summary found yet (b01_summary.tsv).")

# --- Load precomputed 10M summary
if not precomp_summary.exists():
    raise FileNotFoundError(f"Missing precomputed summary: {precomp_summary}")
df_10m = pd.read_csv(precomp_summary, sep="\t", comment="#")
print(f"Loaded PRECOMPUTED summary: {precomp_summary}  ({len(df_10m)} run)")

# --- Combine in-memory (do NOT overwrite student file)
df_all = pd.concat([df_student, df_10m], ignore_index=True)

print("\n=== Combined summary (tail) ===")
display(df_all.tail(10))

# Optional: write a combined file for analysis only
out_dir.mkdir(parents=True, exist_ok=True)
combined_path = out_dir / "b01_summary_with_precomputed.tsv"
df_all.to_csv(combined_path, sep="\t", index=False)
print(f"\nWrote combined summary (analysis-only): {combined_path}")

# --- Load precomputed 10M per-cell table (so they can inspect it)
if not precomp_cells10m.exists():
    raise FileNotFoundError(f"Missing precomputed 10M cell table: {precomp_cells10m}")

df_cells10m = pd.read_csv(precomp_cells10m, sep="\t", comment="#")
print(f"\nLoaded PRECOMPUTED 10M cell table: {precomp_cells10m}  ({len(df_cells10m)} rows)")

print("\n=== 10M cell table (head) ===")
display(df_cells10m)

print("\n=== 10M cell table: ROI row (cell 19) ===")
display(df_cells10m[df_cells10m["cell"] == 19])

## Diagnostics plots (Analog): scaling behavior

In the lectures we discussed two key scaling behaviors expected in
a standard **analog Monte Carlo simulation**.

---

### 1) Statistical scaling

For a fixed physical problem, the relative statistical uncertainty

$$
R \equiv \frac{\sigma}{\mu}
$$

is expected to scale approximately as:

$$
R \propto \frac{1}{\sqrt{N}}
$$

This means that:
- Reducing the uncertainty by a factor of 2
- Requires increasing the number of events by a factor of 4

We will check this by plotting:

- $R$ vs. $1/\sqrt{N}$

A roughly linear trend indicates that the Monte Carlo estimator behaves
as expected from basic statistical theory.

---

### 2) Runtime scaling

For an analog simulation of fixed complexity:

- The wall-clock runtime $T$ is expected to scale approximately linearly with $N$:

$$
T \propto N
$$

We will check this by plotting:

- $T$ vs. $N$

This plot helps quantify the **computational cost** of improving statistical precision
by brute-force increase of the number of events.

---

### What these diagnostics tell us

These plots are **diagnostic tools**, not strict proofs.

They allow us to:
- Verify that the Monte Carlo simulation behaves sensibly
- Estimate how costly it is to reduce the statistical uncertainty
- Anticipate whether increasing $N$ alone is a practical strategy

In the next steps, we will use these trends to estimate the cost
of reaching much tighter uncertainties in deep shielding problems.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Paths you defined earlier (keeping your naming)
build_dir = Path.home() / "CSC26/VR/hands_on_1/build"
out_dir   = build_dir / "out"
pre_dir   = Path.home() / "CSC26/VR/hands_on_1/pcruns"

student_summary  = out_dir / "b01_summary.tsv"
precomp_summary  = pre_dir / "b01_summary_10M.tsv"

def read_summary_tsv(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"Missing summary file: {path}")
    df = pd.read_csv(path, sep="\t")
    df.columns = [c.strip() for c in df.columns]
    return df

# Load and merge (student runs + precomputed 10M)
dfs = []
for p in [student_summary, precomp_summary]:
    try:
        dfs.append(read_summary_tsv(p))
    except Exception as e:
        print(f"[WARN] Could not read {p}: {e}")

if len(dfs) == 0:
    raise RuntimeError("No summary files could be loaded.")

df = pd.concat(dfs, ignore_index=True)

print("Columns found:", list(df.columns))

# Expected columns (from our RunAction design):
col_N    = "Nevt"
col_T    = "time_real_s"
col_R    = "roi_relerr"
col_FOM  = "FOM_1_per_s"

missing = [c for c in [col_N, col_T, col_R] if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns in summary: {missing}")

# Convert key columns to numeric (robust to strings)
for c in [col_N, col_T, col_R]:
    df[c] = pd.to_numeric(df[c], errors="coerce")
if col_FOM in df.columns:
    df[col_FOM] = pd.to_numeric(df[col_FOM], errors="coerce")

# Keep only analog runs if 'mode' exists
if "mode" in df.columns:
    df_analog = df[df["mode"].astype(str).str.strip() == "-1"].copy()
else:
    df_analog = df.copy()

# Compute FOM if not present (or if present but you want to override, comment next if-block and always compute)
if col_FOM not in df_analog.columns:
    df_analog[col_FOM] = 1.0 / (df_analog[col_R]**2 * df_analog[col_T])

# Clean up FOM: set non-physical/undefined values to NaN
# (e.g., R<=0, T<=0, or division artifacts)
bad_fom = (~np.isfinite(df_analog[col_FOM])) | (df_analog[col_FOM] <= 0) | (df_analog[col_R] <= 0) | (df_analog[col_T] <= 0)
df_analog.loc[bad_fom, col_FOM] = np.nan

# Sort by N
df_analog = df_analog.sort_values(col_N)

# ---- Plot 1: R vs 1/sqrt(N) ----
mask_R = (
    np.isfinite(df_analog[col_N]) & (df_analog[col_N] > 0) &
    np.isfinite(df_analog[col_R]) & (df_analog[col_R] > 0)
)
df_R = df_analog.loc[mask_R].copy()

N_R = df_R[col_N].to_numpy(dtype=float)
R   = df_R[col_R].to_numpy(dtype=float)
x   = 1.0 / np.sqrt(N_R)

if len(x) >= 2:
    m, b = np.polyfit(x, R, 1)
    print(f"Fit: R ≈ {m:.3g}*(1/sqrt(N)) + {b:.3g}")
else:
    m, b = np.nan, np.nan
    print("[WARN] Not enough valid points for a linear fit of R vs 1/sqrt(N).")

plt.figure()
plt.plot(x, R, "o", label="Runs")
if np.isfinite(m) and np.isfinite(b):
    xx = np.linspace(x.min()*0.9, x.max()*1.1, 200)
    plt.plot(xx, m*xx + b, "-", label="Linear fit")
plt.xlabel("1/sqrt(N)")
plt.ylabel("Relative uncertainty R = sigma/mu (cell_19)")
plt.title("Analog scaling check: R vs 1/sqrt(N)")
plt.legend()
plt.grid(True)
plt.show()

# ---- Plot 2: T vs N ----
mask_T = (
    np.isfinite(df_analog[col_N]) & (df_analog[col_N] > 0) &
    np.isfinite(df_analog[col_T]) & (df_analog[col_T] > 0)
)
df_T = df_analog.loc[mask_T].copy()

N_T = df_T[col_N].to_numpy(dtype=float)
T   = df_T[col_T].to_numpy(dtype=float)

if len(N_T) >= 2:
    mT, bT = np.polyfit(N_T, T, 1)
    print(f"Fit: T ≈ {mT:.3e}*N + {bT:.3g} (seconds)")
else:
    mT, bT = np.nan, np.nan
    print("[WARN] Not enough valid points for a linear fit of T vs N.")

plt.figure()
plt.plot(N_T, T, "o", label="Runs")
if np.isfinite(mT) and np.isfinite(bT):
    NN = np.linspace(N_T.min()*0.9, N_T.max()*1.1, 200)
    plt.plot(NN, mT*NN + bT, "-", label="Linear fit")
plt.xlabel("N events")
plt.ylabel("Wall time T (s)")
plt.title("Runtime scaling check: T vs N")
plt.legend()
plt.grid(True)
plt.show()

# Optional: Show a compact table (including NaNs so students can see what's undefined)
out_cols = [c for c in ["mode", col_N, col_T, col_R] if c in df_analog.columns]
out = df_analog[out_cols].copy()
out["R_%"] = 100.0*out[col_R]
print("\nSummary (analog runs):")
display(out)

## Diagnostics (analog runs): scaling of uncertainty and runtime

You have just produced two diagnostic plots for **analog** runs in the ROI (`cell_19`):

1) **Relative uncertainty** $R$ vs $1/\sqrt{N}$  (statistical scaling check)  
2) **Wall time** $T$ vs $N$ (runtime scaling check)

These are *sanity diagnostics*: they help us see whether the simulation behaves like a standard Monte Carlo estimator and whether brute-force scaling is a plausible strategy.

---

### 1) Statistical scaling in the ROI: does $R$ decrease with $N$?

- As $N$ increases, does $R$ decrease?  
- Does the plot of $R$ vs $1/\sqrt{N}$ look **roughly linear** (same trend, not a perfect line)?

**Quantitative check (order-of-magnitude, not exact):**  
If $N$ increases by a factor of 10, the *ideal* Monte Carlo expectation is:

$$
R \propto \frac{1}{\sqrt{N}}
\quad\Rightarrow\quad
\frac{R(N)}{R(10N)} \approx \sqrt{10} \approx 3.16
$$

In deep shielding, you should not expect the factor to match $3.16$ perfectly, because the ROI is dominated by **rare histories** at moderate $N$.  
So here we only ask:

- Is the reduction **clearly of the right scale** (e.g. a factor $\sim 2$–$4$)?  
- Does increasing $N$ keep pushing $R$ down in a consistent way?

Write one sentence summarizing what you observe.

---

### 2) Runtime scaling: does $T$ increase roughly proportionally to $N$?

- Does the wall time increase as you increase $N$?  
- Does $T$ vs $N$ look **approximately linear** over the runs you have?

A practical check is to compare time ratios:

- If $N$ increases by $10\times$, is $T$ also of the same order ($\sim 10\times$)?

**About the line fit:**  
You do not need a strict criterion. We simply want to see whether the fitted line follows the data trend (no obvious curvature or dramatic departures).

---

### 3) Practical conclusion: cost vs precision

Using both plots together, answer:

- Does “just increase $N$” look practical for reaching **$R < 10\%$** in `cell_19` within a hands-on session?
- What is the main limitation: time, or slow uncertainty reduction?

Write a short conclusion (2–4 lines).

---
## From scaling trends to estimator reliability (near vs ROI)

The diagnostics you just did tell us something important:

- In the **ROI (`cell_19`)**, increasing $N$ tends to reduce $R$, and runtime tends to grow with $N$.
- This is consistent with the expected *Monte Carlo scaling behavior*.

However, in deep shielding there is one extra complication:

> Even if $R$ decreases with $N$, the **mean estimate** in a rare-event region can still change noticeably at moderate statistics,
because new rare contributions start to appear only when $N$ becomes large enough.

So we now ask a more practical reliability question:

> **Which parts of the geometry look “settled” already, and which parts still fluctuate strongly with $N$?**

To answer that, we compare:
- a **near-source cell** with abundant statistics (`cell_03`), and
- the **deep ROI** (`cell_19`), dominated by rare histories.

This comparison will help us interpret the ROI results and motivate why variance reduction is needed.

---
## Stability check: a near cell vs the deep-penetration ROI

Scaling laws describe how uncertainty behaves *on average*.  
They do **not** tell us whether the **mean estimate** is already “settled” at the statistics we can afford.

In deep shielding:

- **`cell_03` (near-source)** receives many contributions.  
  As $N$ increases, the mean should change little, while uncertainty shrinks.

- **`cell_19` (ROI)** is dominated by rare histories.  
  At moderate $N$, the mean can shift noticeably as new rare contributions start to appear.

This is not a bug — it is exactly the motivation for variance reduction.

---

### Task

Using the per-run cell tables (`b01_cells_*.tsv`), compare `cell_03` and `cell_19`
across the analog runs you have available (e.g. $10^5$, $10^6$, precomputed $10^7$).

---

### Questions (short answers)

1. Does **Mean(SLW)** in `cell_03` change significantly as $N$ increases?  
   What does this tell you about estimator stability when statistics are abundant?

2. Does **Mean(SLW)** in `cell_19` change significantly as $N$ increases?  
   Why is this expected for rare-event regions?

3. Compare the **relative uncertainty** in `cell_03` and `cell_19`.  
   What physical and statistical effects explain the difference?

4. Which cell would you trust more as a **reference region** when checking consistency across different sampling strategies later (analog vs VR)?

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

# --- Paths (as defined earlier in your notebook) ---
build_dir = Path.home() / "CSC26/VR/hands_on_1/build"
out_dir   = build_dir / "out"
pre_dir   = Path.home() / "CSC26/VR/hands_on_1/pcruns"

# Helper: find the per-run tables (student runs + precomputed 10M)
student_tables = sorted(out_dir.glob("b01_cells_mode-1_run*_N*.tsv"))
precomp_tables = sorted(pre_dir.glob("b01_cells_mode-1_run*_N*.tsv"))  # include 10M table here

tables = student_tables + precomp_tables

if len(tables) == 0:
    raise RuntimeError("No per-run cell tables found (b01_cells_mode-1_run*_N*.tsv).")

print("Per-run tables found:")
for p in tables:
    print(" -", p)

def parse_N_from_filename(path: Path) -> int:
    # expected pattern: ..._N100000.tsv
    s = path.name
    i = s.rfind("_N")
    j = s.rfind(".tsv")
    return int(s[i+2:j])

def load_cells_table(path: Path) -> pd.DataFrame:
    # File has comment lines starting with '#'
    df = pd.read_csv(path, sep="\t", comment="#")
    # normalize column names (robust to small header edits)
    df.columns = [c.strip() for c in df.columns]
    return df

# Choose cells for stability comparison
CELL_NEAR = "cell_03"
CELL_ROI  = "cell_19"

rows = []
for p in tables:
    N = parse_N_from_filename(p)
    df = load_cells_table(p)

    # Expect columns similar to: cellName, SLW_mean_mm, SLW_stderr_mm, etc.
    # Your RunAction writes:
    # "cell  cellName  TrEntering Population AvTrWgt SL_mean_mm SL_stderr_mm SLW_mean_mm SLW_stderr_mm"
    if "cellName" not in df.columns:
        raise ValueError(f"Missing 'cellName' column in {p}")

    for cname in [CELL_NEAR, CELL_ROI]:
        sub = df[df["cellName"] == cname]
        if len(sub) != 1:
            raise ValueError(f"Could not find exactly one row for {cname} in {p}")
        r = sub.iloc[0]

        mean = float(r["SLW_mean_mm"])
        se   = float(r["SLW_stderr_mm"])
        rel  = (se/mean) if mean > 0 else np.nan

        rows.append({
            "Nevt": N,
            "cell": cname,
            "SLW_mean_mm": mean,
            "SLW_stderr_mm": se,
            "rel_uncertainty": rel,
        })

stab = pd.DataFrame(rows).sort_values(["cell", "Nevt"]).reset_index(drop=True)

# Print a compact stability table
stab_show = stab.copy()
stab_show["rel_%"] = 100.0 * stab_show["rel_uncertainty"]
print("\nStability comparison (SLW):")
display(stab_show[["cell", "Nevt", "SLW_mean_mm", "SLW_stderr_mm", "rel_%"]])

# Optional: quantify "stability" as fractional change of the mean vs the previous N
stab["frac_change_vs_prev"] = np.nan
for cell in [CELL_NEAR, CELL_ROI]:
    m = stab.loc[stab["cell"] == cell, "SLW_mean_mm"].to_numpy()
    n = stab.loc[stab["cell"] == cell, "Nevt"].to_numpy()
    # fractional change: |m_i - m_{i-1}| / m_{i-1}
    fc = np.full_like(m, np.nan, dtype=float)
    for i in range(1, len(m)):
        if m[i-1] != 0:
            fc[i] = abs(m[i] - m[i-1]) / abs(m[i-1])
    stab.loc[stab["cell"] == cell, "frac_change_vs_prev"] = fc

stab_fc = stab.copy()
stab_fc["frac_change_%"] = 100.0 * stab_fc["frac_change_vs_prev"]
print("\nFractional change of Mean(SLW) vs previous N (diagnostic for stability):")
display(stab_fc[["cell", "Nevt", "SLW_mean_mm", "frac_change_%"]])

## How far can we go with analog simulation?

So far, you have observed (for the ROI `cell_19`) that:

- The relative uncertainty decreases roughly like
  $$
  R \propto \frac{1}{\sqrt{N}}
  $$
- The wall time increases approximately linearly with the number of events,
  $$
  T \propto N
  $$

We now ask a practical question:

> **If we keep increasing $N$, how expensive is it to reach the target precision in `cell_19`?**

---

### Step 1 — Organize your runs (no plotting)

Fill a small table using the numbers printed in the ROI summary:

- $N$
- $R$ (relative uncertainty) in `cell_19`
- $T$ (wall time)

You should have at least:
- your runs (e.g. $10^5$, $10^6$)
- plus the precomputed analog run (e.g. $10^7$)

---

### Step 2 — Pick one reference run

Choose **one** run as reference $(N_0, R_0, T_0)$.

**Recommendation:** use the run with the **largest $N$** available (it is usually the most reliable).

---

### Step 3 — Estimate $N$ needed to reach the target

Assuming the ideal Monte Carlo scaling holds in the ROI,
$$
R \approx R_0 \sqrt{\frac{N_0}{N}},
$$
then the statistics required to reach a target $R_\text{target}$ are:
$$
\frac{N_\text{target}}{N_0} \approx \left(\frac{R_0}{R_\text{target}}\right)^2.
$$

Compute (order of magnitude is enough):
- $N_{10\%}$ for $R_\text{target}=10\%$.

---

### Step 4 — Estimate the runtime cost

Assuming runtime is roughly linear in $N$,
$$
T_\text{target} \approx T_0 \frac{N_\text{target}}{N_0}.
$$

Estimate:
- $T_{10\%}$ (seconds or minutes)

---

### Quick check

Is reaching $R<10\%$ by brute-force analog simulation:

- feasible within a hands-on session?
- feasible as a routine strategy?

Explain in **one sentence**.

## Switching on Variance Reduction: Importance Biasing (`mode = 0`)

So far we ran **analog Monte Carlo** (`mode = -1`).  
We saw that in the **region of interest (ROI)**, **`cell_19`**, the relative uncertainty decreases too slowly with \(N\), so reaching the **10% goal** by brute force is impractical in a hands-on setting.

Now we will activate **variance reduction** using **importance biasing**.

---

### What changes (and what does NOT)

✅ **Geometry stays exactly the same**  
Same world, same cylindrical shielding setup, same segmentation into `cell_00` … `cell_19`.

✅ **Materials stay exactly the same**  
Same concrete shield (and the same non-shield regions as defined in the exercise).

✅ **Physics stays exactly the same**  
Same physics list: **`FTFP_BERT_HP`** (same processes, same cuts).

✅ **Scoring stays exactly the same**  
Same estimators and the same output table + ROI summary for `cell_19`.

❗ The only change is the **transport strategy**: we guide particle histories using **geometric importances**.

---

### The importance map used in this exercise

Inside the shield, importances increase **by a factor of 2 per cell**:

$$
\frac{I_{i+1}}{I_i} = 2
\quad\Rightarrow\quad
I_i = 2^{(i-1)} \;\;\text{for}\;\; i=1,2,\dots,18
$$

So each step deeper is “twice as important” as the previous one.

You will actually *see* the assigned importances printed in the output when `mode = 0`:

### Where does the importance map come from?

In this exercise, the importance values are defined directly in the Geant4 example source code:

- `B01DetectorConstruction.cc`
- method: `CreateImportanceStore()`

**What it does:**
- The code iterates over the longitudinal scoring volumes (`fPhysicalVolumeVector`)
- It assigns importances that grow as powers of two along the shield:
  $$
  I_n = 2^n
  $$
  (with $n$ increasing as we move deeper in $+z$)

**What does this mean for this hands-on:**
- This importance gradient is what triggers **splitting / Russian roulette**
- As a result, particle **weights change** and deep regions receive **more contributing histories**

> You do *not* need to read the C++ to complete the hands-on.  
> If you are curious, we can open the relevant function during the session break.

## Confirming the importance map

When running with `mode = 0`, the simulation prints the **importance values assigned to each cell**.

👉 This printout confirms that the simulation is running with the **intended importance map**, and that variance reduction is effectively enabled.

---

## How Geant4 uses importances  
*(splitting and Russian roulette)*

When a particle crosses a boundary from a region with importance  
$ I_{\text{old}} $ to $ I_{\text{new}} $, Geant4 modifies the transport as follows.

### Case A: $ I_{\text{new}} > I_{\text{old}} $ — going deeper into the shield

Geant4 increases sampling through **track splitting**:

- The particle is replaced by several copies.
- Each copy carries a **reduced statistical weight**.
- The **expected contribution remains unbiased** when weighted estimators are used.

This increases the number of particle histories reaching deep regions.

---

### Case B: $ I_{\text{new}} < I_{\text{old}} $ — moving back toward less important regions

Geant4 applies **Russian roulette**:

- The particle may be killed with some probability.
- If it survives, its weight is increased accordingly.
- CPU time is not wasted tracking particles in regions that are unimportant for the ROI.

---

## What should change in the output

Compared to the analog runs, you should now observe:

- **Higher population in deep cells**, including the ROI (`cell_19`).
- **Average track weight (`Av.Tr.WGT`) decreasing with depth**  
  (splitting → more tracks with smaller weights).
- **`SL` and `SLW` are no longer equal**. **Why? 🤔** 

The quantity of interest remains the **ROI summary for `cell_19`**, in particular:
- Mean(SLW)
- StdErr(SLW)
- Relative error (%)
- Execution time
- Figure of Merit (FOM)

---

## A note on stability

Also monitor a **near-source cell** (e.g. `cell_03`):

- Its mean value should remain **stable** when switching from analog to biased transport.
- This will later help you distinguish:
  - genuine variance reduction,
  - from potential pathologies introduced by biased sampling.

---

## Run: Importance biasing enabled

Now run the simulation with:

- **Mode:** `mode = 0` (importance biasing ON)
- **Number of events:**  
  $$N = 100{,}000$$

### Your task

Compare this run against the **analog run with the same $N$**:

- Per-cell tables:
  - Population
  - Av.Tr.WGT
  - SL vs SLW
- ROI summary for `cell_19`

Quantify:
- how much the **relative uncertainty** decreases,
- how the **execution time** changes,
- and how the **overall efficiency** improves.

In [ ]:
# Non-analog run: N = 100,000 events

import os, subprocess
from pathlib import Path

base_dir  = Path.home() / "CSC26/VR/hands_on_1"
build_dir = base_dir / "build"
exe = build_dir / "exampleB01"  # estando en build/
mode = 0
N = 100000

env = os.environ.copy()
env["B01_MODE"] = str(mode)

subprocess.run([str(exe), str(mode), str(N)], env=env, check=True)

print("\nRun completed.")
print("Check the screen output and the files written in build/out/")

## Inspecting the per-cell table (Variance Reduction vs Analog, same $N$)

In the previous section you ran an **analog simulation** (`mode = -1`) with  
$N = 100{,}000$ events.

You have now run the **same simulation** with **importance biasing enabled**  
(`mode = 0`) and the **same number of events**.

✅ **Nothing physical changed**  
- Same geometry  
- Same materials  
- Same source  
- Same physics list  
- Same scoring definitions  

❗ **Only the transport strategy changed**  
Geant4 now uses **splitting and Russian roulette**, driven by **geometric importances**.

Your task in this section is to inspect the **per-cell table** and understand:
- what changed,
- what stayed the same,
- and *why* these changes occur.

---

### Focus cells for inspection

To keep the analysis manageable, focus on two representative cells:

- **`cell_03`**  
  A *shallow* cell, close to the source.  
  Many particle histories contribute here, so estimates should be **stable**.

- **`cell_19`**  
  The **deep-penetration region of interest (ROI)**.  
  Very few analog histories reach this cell, which is exactly why variance reduction is needed.

You should still glance at the **full per-cell table**, but your answers below
should explicitly refer to **`cell_03`** and **`cell_19`**.

---

### Questions (short, qualitative answers)

1. **Population**
   - How does the `Population` in `cell_19` compare between analog and VR runs?
   - Why does importance biasing change this quantity?

2. **Average track weight**
   - How does `Av.Tr.WGT` behave as a function of depth in the VR run?
   - Why is this behavior expected when splitting is active?

3. **SL vs SLW**
   - Are `SL` and `SLW` still equal in the VR run?
   - Which one is the **physically correct estimator** when variance reduction is enabled, and why?

4. **Near vs deep cell**
   - Does `cell_03` show large changes between analog and VR?
   - Why should (or shouldn’t) this cell be sensitive to the transport strategy?

---

### Takeaway

This table-level inspection helps you verify that:
- variance reduction is **working as intended**,
- deep regions receive more statistical weight,
- and unbiased estimators (`SLW`) must be used once particle weights are no longer uniform.

In the next step, you will quantify how these changes affect **uncertainty, runtime, and efficiency** in the ROI.

In [ ]:
# --- Load and display the per-cell tables for N=100k (mode = -1 and mode = 0) ---
# This cell assumes you already defined:
#   build_dir = Path.home() / "CSC26/VR/hands_on_1/build"
#   out_dir   = build_dir / "out"

import pandas as pd
from pathlib import Path

N_TARGET = 100000
MODES = [-1, 0]

def find_cells_table(out_dir: Path, mode: int, N: int) -> Path:
    """
    Find the per-run table file:
      out/b01_cells_mode{mode}_run{runID}_N{Nevt}.tsv
    We don't assume runID, we just pick the newest match.
    """
    pattern = f"b01_cells_mode{mode}_run*_N{N}.tsv"
    matches = sorted(out_dir.glob(pattern), key=lambda p: p.stat().st_mtime, reverse=True)
    if not matches:
        raise FileNotFoundError(
            f"No per-cell table found for mode={mode}, N={N} in {out_dir}\n"
            f"Expected something like: {pattern}\n"
            f"Tip: did you run with the env var set? e.g. B01_MODE={mode}"
        )
    return matches[0]

def read_cells_table(path: Path) -> pd.DataFrame:
    # Our file has a header row and may have comment lines starting with '#'
    df = pd.read_csv(path, sep="\t", comment="#")
    df.columns = [c.strip() for c in df.columns]
    return df

tables = {}

print(f"Looking for per-cell tables in: {out_dir}")
for mode in MODES:
    path = find_cells_table(out_dir, mode, N_TARGET)
    df = read_cells_table(path)
    tables[mode] = (path, df)
    print(f"[OK] mode={mode}: {path.name}")

# Display full tables (scrollable) in the notebook
for mode in MODES:
    path, df = tables[mode]
    print(f"\n=== Per-cell table (mode={mode}, N={N_TARGET}) ===")
    display(df)

### Columns to inspect and interpret

For **each** of the following columns, compare **mode = -1** vs **mode = 0** (same N = 100k):

1. **`Tr.Entering`**  
2. **`Population`**  
3. **`Av.Tr.WGT`**  
4. **`SL`** and **`Sigma(SL)`**  
5. **`SLW`** and **`Sigma(SLW)`**

---

### Write your observations (space for answers)

Fill the table below using short bullet points.

| Quantity | `cell_03`: what changes in mode 0 vs mode -1? | `cell_19`: what changes in mode 0 vs mode -1? | Why (one sentence) |
|---|---|---|---|
| Tr.Entering |  |  |  |
| Population |  |  |  |
| Av.Tr.WGT |  |  |  |
| SL & Sigma(SL) |  |  |  |
| SLW & Sigma(SLW) |  |  |  |

---

### Key conceptual questions

1) **In analog runs (mode = -1), why do we typically observe `SL ≈ SLW`?**  
**Answer:**  

2) **In VR runs (mode = 0), which estimator should be used as the unbiased result: `SL` or `SLW`? Why?**  
**Answer:**  

3) **Based on the table, does VR seem to “move statistics” toward deep cells? Point to two specific columns that support your answer.**  
**Answer:**  

---

### Takeaway (one line)

Write one sentence summarizing what you learned from the table inspection:

**Takeaway:** ___________________________________________

## Evaluating variance reduction: three essential checks

So far, you have:

- Understood the physical problem and the observable,
- Run **analog** simulations and observed slow convergence in the deep ROI,
- Enabled **importance biasing** and inspected how the per-cell statistics change.

At this point, we move from *exploration* to **evaluation**.

Whenever a variance reduction (VR) technique is introduced, it must be assessed carefully.
Improving statistical precision alone is **not sufficient**.

In this hands-on, we will evaluate importance biasing using **three fundamental checks**,
which mirror best practice in real Monte Carlo work:

---

### The three checks

**1. Accuracy (unbiasedness)**  
Does variance reduction preserve the *physical expectation value* of the observable?

> The mean value must remain consistent with the analog result within statistical uncertainty.

---
**2. Efficiency (cost vs precision)**  
Does variance reduction reduce uncertainty *fast enough* to justify its additional cost?

> This is where we quantify whether VR actually pays off in practice.


---

**3. Stability (pathologies and weight behavior)**  
Does variance reduction introduce rare but extreme contributions that dominate the estimator?

> We want to avoid estimators controlled by a handful of pathological events.

---

In the next cells, you will perform these three checks **step by step**, using the data
you have already produced.

We start with the most fundamental one:

➡️ **Accuracy — does variance reduction preserve the mean?**

## Check 1 — Accuracy: does variance reduction preserve the mean?

So far, you have inspected **how the per-cell statistics change** when importance biasing is enabled:
- more tracks reach deep cells,
- particle weights are redistributed,
- raw and weighted estimators no longer coincide.

Now we move from *qualitative inspection* to a **quantitative check**.

---

### Why this check matters

Variance reduction techniques are only useful if they satisfy a strict requirement:

> **They must not change the physical expectation value of the observable.**

Importance biasing is allowed to:
- change how often certain histories are sampled,
- introduce splitting and Russian roulette,
- redistribute statistical weights,

but it must **not bias the final result**.

This property is called **accuracy** or **unbiasedness**.

Because we work with *finite statistics*, two runs (analog and VR) will **never give exactly the same number**.
What matters is whether the difference is **consistent with statistical fluctuations**.

---

### Comparing two estimates with uncertainties

Let:
- $\mu_{-1}$ = estimate from the **analog** run (`mode = -1`)
- $\mu_{0}$  = estimate from the **variance-reduced** run (`mode = 0`)
- $\sigma_{-1}$ and $\sigma_{0}$ = the **standard errors** of the estimator  
  (the values printed as `Sigma(SLW)`)

We first compute the difference:
$$
\Delta = \mu_{0}-\mu_{-1}
$$

If the two runs are statistically independent, the uncertainty on this difference is:
$$
\sigma_\Delta = \sqrt{\sigma_0^2 + \sigma_{-1}^2}
$$

**Why the square root?**  
Because variances add for independent random variables:
$$
\mathrm{Var}(X-Y)=\mathrm{Var}(X)+\mathrm{Var}(Y)
$$

We then define a normalized deviation (a “z-score”):
$$
z = \frac{\mu_{0}-\mu_{-1}}{\sqrt{\sigma_0^2+\sigma_{-1}^2}}
$$

---

### How to interpret the result

As a rule of thumb:
- $|z|\lesssim 2$:  
  The two results are **statistically compatible** (no evidence of bias).
- $|z|\gg 2$:  
  Potential problem — could indicate bias, underestimated uncertainties, or insufficient statistics.

This is **not** a strict hypothesis test, but a practical diagnostic.

---

### What to do

1. Compare **SLW** (not SL) between `mode = -1` and `mode = 0`.  
2. Start with a **near-source cell** such as `cell_03`, where analog statistics are good.  
3. Inspect the **trend vs depth**:
   - SLW should decrease smoothly as we move deeper into the shield.
   - No systematic shift between analog and VR should appear.

👉 At this stage, focus on **consistency**, not exact numerical equality.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# expects: tables dict from your previous cell:
# tables[mode] = (path, df)
# and df has at least: cellName, SLW_mean_mm, SLW_stderr_mm

cell_acc = "cell_03"  # near-source cell for good analog stats

df_m1 = tables[-1][1].copy()
df_0  = tables[0][1].copy()

# --- z-score at a reference cell ---
m1 = df_m1.set_index("cellName")
m0 = df_0.set_index("cellName")

mu_m1 = float(m1.loc[cell_acc, "SLW_mean_mm"])
se_m1 = float(m1.loc[cell_acc, "SLW_stderr_mm"])
mu_0  = float(m0.loc[cell_acc, "SLW_mean_mm"])
se_0  = float(m0.loc[cell_acc, "SLW_stderr_mm"])

sigma_delta = np.sqrt(se_0**2 + se_m1**2)
z = (mu_0 - mu_m1) / sigma_delta if sigma_delta > 0 else np.nan

print(f"Accuracy check at {cell_acc} (N=100k):")
print(f"  mode=-1: mean(SLW)={mu_m1:.6g} mm, stderr={se_m1:.6g} mm")
print(f"  mode= 0: mean(SLW)={mu_0:.6g} mm, stderr={se_0:.6g} mm")
print(f"  Δ = {mu_0-mu_m1:.6g} mm")
print(f"  σ_Δ = sqrt(σ0^2+σ-1^2) = {sigma_delta:.6g} mm")
print(f"  z = Δ/σ_Δ = {z:.3f}   (|z| ≲ 2 → compatible)")

# --- helper: depth index from "cell_XX" ---
def cell_index(name: str) -> int:
    return int(name.split("_")[1])

df_m1["idx"] = df_m1["cellName"].apply(cell_index)
df_0["idx"]  = df_0["cellName"].apply(cell_index)

df_m1 = df_m1.sort_values("idx")
df_0  = df_0.sort_values("idx")

# --- Plot SLW mean vs depth for both modes ---
plt.figure()
plt.plot(df_m1["idx"], df_m1["SLW_mean_mm"], "o-", label="mode = -1 (analog), N=100k")
plt.plot(df_0["idx"],  df_0["SLW_mean_mm"],  "o-", label="mode = 0 (importance), N=100k")
plt.xlabel("Cell index (depth)")
plt.ylabel("Mean(SLW) [mm]")
plt.title("Accuracy diagnostic: SLW vs depth (same N, different modes)")
plt.yscale("log")  # deep shielding trend is easier to see on a log scale
plt.grid(True)
plt.legend()
plt.show()

# Optional: also visualize uncertainties (stderr) vs depth
plt.figure()
plt.plot(df_m1["idx"], df_m1["SLW_stderr_mm"], "o-", label="mode = -1 stderr")
plt.plot(df_0["idx"],  df_0["SLW_stderr_mm"],  "o-", label="mode = 0 stderr")
plt.xlabel("Cell index (depth)")
plt.ylabel("StdErr(SLW) [mm]")
plt.title("Uncertainty vs depth (same N, different modes)")
plt.yscale("log")
plt.grid(True)
plt.legend()
plt.show()

## From correctness to performance

In **Check 1**, we verified something fundamental:
importance biasing does **not** distort the physical mean of the observable.

Once accuracy is established, the next question is purely practical:

> **Is variance reduction actually worth it?**

In real Monte Carlo work, we rarely care only about correctness.
We care about **how much computer time is needed** to reach a given precision,
especially in regions dominated by rare events.

To answer that question quantitatively, we now introduce a standard performance metric:
the **Figure of Merit (FOM)**.

## Check 2 — Efficiency (FOM improvement where it matters)

**Goal:** show that importance biasing gives a higher **Figure of Merit (FOM)** in the region of interest (ROI).

Recall the definition used by the code (for `cell_19`, using SLW):
$$
\mathrm{FOM}=\frac{1}{R^2\,T}
\quad\text{with}\quad
R=\frac{\sigma}{\mu}
$$
- A good VR method typically **increases FOM**, even if it also increases run time \(T\),
  because it reduces the relative uncertainty \(R\) a lot.

### What to compare
- **Analog (mode = -1), N = 10,000,000**  *(precomputed in SWAN)*
- **Importance (mode = 0), N = 100,000** *(you just ran it)*

Write down:
- \(R\), \(T\), and FOM for both cases.
- The ratio:
$$
\frac{\mathrm{FOM}_{\mathrm{VR}}}{\mathrm{FOM}_{\mathrm{analog}}}
$$
Interpretation: “How many times more efficient is VR in the ROI?”

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# Paths from earlier
build_dir = Path.home() / "CSC26/VR/hands_on_1/build"
out_dir   = build_dir / "out"
pre_dir   = Path.home() / "CSC26/VR/hands_on_1/pcruns"

student_summary  = out_dir / "b01_summary.tsv"
precomp_summary  = pre_dir / "b01_summary_10M.tsv"

def read_summary(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, sep="\t")
    df.columns = [c.strip() for c in df.columns]
    return df

df_sum = pd.concat([read_summary(student_summary),
                    read_summary(precomp_summary)], ignore_index=True)

# Keep only rows we need
# We assume summary columns exist: mode, Nevt, time_real_s, roi_relerr, FOM_1_per_s
needed = ["mode", "Nevt", "time_real_s", "roi_relerr", "FOM_1_per_s"]
missing = [c for c in needed if c not in df_sum.columns]
if missing:
    raise ValueError(f"Missing columns in summary: {missing}")

def pick_row(df, mode, Nevt):
    sub = df[(df["mode"].astype(str).str.strip() == str(mode)) & (df["Nevt"] == Nevt)].copy()
    if len(sub) == 0:
        raise RuntimeError(f"No summary row found for mode={mode}, Nevt={Nevt}")
    # If multiple, take the last one appended
    return sub.iloc[-1]

row_vr   = pick_row(df_sum, 0, 100000)
row_10m  = pick_row(df_sum, -1, 10000000)

def fmt_row(r):
    R = float(r["roi_relerr"])
    T = float(r["time_real_s"])
    F = float(r["FOM_1_per_s"])
    return R, T, F

R_vr,  T_vr,  F_vr  = fmt_row(row_vr)
R_10m, T_10m, F_10m = fmt_row(row_10m)

print("Efficiency check (ROI = cell_19, using SLW):")
print(f"  VR   mode=0,   N=1e5 : R={100*R_vr:.3g} %,  T={T_vr:.3g} s,  FOM={F_vr:.3g} 1/s")
print(f"  Analog mode=-1, N=1e7 : R={100*R_10m:.3g} %, T={T_10m:.3g} s, FOM={F_10m:.3g} 1/s")
print()
print(f"  FOM improvement factor = FOM_VR / FOM_Analog = {F_vr/F_10m:.3g}")

## Why a stability check is still needed

So far we have shown that importance biasing:

- Preserves the physical mean (accuracy)
- Improves performance in the ROI (efficiency)

However, **neither of these guarantees a healthy Monte Carlo simulation**.

A variance reduction setup can be:
- Unbiased
- Efficient on average
  
and still be **dangerous**, if it relies on very rare events with extremely large weights.

Such pathologies often reveal themselves only when inspecting
**weight behavior and consistency across regions**.

This final check focuses on **stability**, using simple diagnostics available in this exercise.

## Check 3 — Stability (weight behavior + diagnostic consistency)

**Goal:** detect "pathological" VR configurations that create rare **huge weights** ("spikes"),
which can ruin statistics even if the mean is unbiased.

In our simplified setup, we use two practical diagnostics:

### (A) Weight trend with depth
Look at `Av.Tr.WGT` versus cell index:
- In mode = 0, as importances increase with depth, splitting produces **more tracks with smaller weights**.
- So `Av.Tr.WGT` should generally **decrease** with depth.

### (B) Near-source stability across modes
Check that in near-source cells (e.g. `cell_02`–`cell_05`) the results are well-behaved:
- `SLW_mean_mm` should remain consistent with analog within uncertainties (accuracy-like check)
- `Sigma(SLW)` should not explode.

> In a full production exercise, we would also histogram weights and look for spikes.
> Here we approximate this with the *average weight trend* and near-source consistency.

Write down:
- `Av.Tr.WGT` in `cell_03` and in the deepest shield cell (e.g. `cell_18`) for mode 0.
- Is the trend monotonic decreasing? Any weird jumps?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Use the per-cell tables we already loaded for N=100k
df_m1 = tables[-1][1].copy()
df_0  = tables[0][1].copy()

# Extract cell index from the name "cell_XX"
def cell_index(name):
    return int(name.split("_")[1])

df_0["idx"] = df_0["cellName"].apply(cell_index)
df_0 = df_0.sort_values("idx")

# --- Plot average weight vs depth (mode 0) ---
plt.figure()
plt.plot(df_0["idx"], df_0["AvTrWgt"], "o-")
plt.yscale('log')
plt.xlabel("Cell index (depth)")
plt.ylabel("Av.Tr.WGT (mode 0)")
plt.title("Stability diagnostic: average weight vs depth (importance biasing)")
plt.grid(True)
plt.show()

# --- Print a quick table for near-source cells (02-05) comparing mode -1 vs 0 ---
focus = [f"cell_{i:02d}" for i in range(2, 6)]

cmp_cols = ["SLW_mean_mm", "SLW_stderr_mm", "AvTrWgt", "Population", "TrEntering"]
cmp_cols = [c for c in cmp_cols if c in df_0.columns]

m1 = df_m1.set_index("cellName").loc[focus, cmp_cols].add_prefix("mode_-1_")
m0 = df_0.set_index("cellName").loc[focus, cmp_cols].add_prefix("mode_0_")
cmp = pd.concat([m1, m0], axis=1)

print("Near-source cells consistency (N=100k):")
display(cmp)

Up to now, we have evaluated variance reduction using three qualitative checks:

1. **Accuracy** — does it preserve the physical mean?
2. **Efficiency** — does it reduce uncertainty faster than brute force?
3. **Stability** — does it behave well at the event and weight level?

We now summarize these conclusions **quantitatively**, using the same metrics
you have already seen: relative uncertainty, runtime, and Figure of Merit (FOM).

## Final quantitative comparison

We now quantify *why* variance reduction exists by comparing the
**region of interest (`cell_19`)** using three metrics:

- Relative uncertainty: $R = \sigma/\mu$
- Wall time: $T$ (seconds)
- Figure of Merit:
$$
\mathrm{FOM} = \frac{1}{R^2\,T}
$$

**Important:** a higher FOM means *more precision per unit of CPU time*.

---

### (a) Fair comparison: same statistics, different transport  
**Analog 100k vs VR 100k**

This is the cleanest possible comparison:

- Same geometry, materials, physics, scoring, and machine
- Same number of events ($N = 100{,}000$)
- **Only the transport strategy changes**

**Your task:**
1. Extract $R$, $T$, and FOM for:
   - Analog 100k
   - VR 100k
2. Compute the ratios:
   - $R_{\text{analog}} / R_{\text{VR}}$
   - $T_{\text{VR}} / T_{\text{analog}}$
   - $\mathrm{FOM}_{\text{VR}} / \mathrm{FOM}_{\text{analog}}$

Interpretation:
- How much uncertainty reduction do you get?
- What is the runtime penalty?
- Is the trade-off worth it?

---

### (b) Practical comparison: same environment, different $N$  
**Analog 10M vs VR 100k**

This is the comparison that shows why VR is necessary.

- Analog 10M was run in the **same SWAN environment**
- It is provided precomputed because it is too slow to run live

**Your task:**
1. Compare:
   - Analog 10M
   - VR 100k
2. Compute:
   - Event ratio: $N_{10\text{M}} / N_{100\text{k}}$
   - $R_{10\text{M}} / R_{\text{VR}}$
   - $T_{10\text{M}} / T_{\text{VR}}$
   - $\mathrm{FOM}_{\text{VR}} / \mathrm{FOM}_{10\text{M}}$

Interpretation:
- How many more events does analog need to reach similar precision?
- How many times more efficient is VR in the ROI?

---

➡️ Run the next code cell to load the summary files and compute these comparisons.

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

build_dir = Path.home() / "CSC26/VR/hands_on_1/build"
out_dir   = build_dir / "out"
pre_dir   = Path.home() / "CSC26/VR/hands_on_1/pcruns"

student_summary  = out_dir / "b01_summary.tsv"
precomp_summary  = pre_dir / "b01_summary_10M.tsv"

def read_summary_tsv(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"Missing summary file: {path}")
    df = pd.read_csv(path, sep="\t")
    df.columns = [c.strip() for c in df.columns]
    return df

dfs = [read_summary_tsv(student_summary), read_summary_tsv(precomp_summary)]
df = pd.concat(dfs, ignore_index=True)

# Expected columns from RunAction
col_mode = "mode"
col_N    = "Nevt"
col_T    = "time_real_s"
col_R    = "roi_relerr"
col_F    = "FOM_1_per_s"

# Basic checks
for c in [col_N, col_T, col_R]:
    if c not in df.columns:
        raise ValueError(f"Missing required column: {c}")

# Compute FOM if not present
if col_F not in df.columns:
    df[col_F] = 1.0 / (df[col_R]**2 * df[col_T])

# Keep only rows with finite R and T
df = df[np.isfinite(df[col_R]) & np.isfinite(df[col_T])].copy()

# Show a compact table (ROI only)
show_cols = [c for c in ["mode", col_N, col_T, col_R, col_F] if c in df.columns]
roi = df[show_cols].copy()
roi["R_%"] = 100.0 * roi[col_R]
roi = roi.sort_values([col_mode, col_N], key=lambda s: s.astype(str))

print("ROI summary rows available:")
display(roi)

In [ ]:
def pick_run(df, mode, N):
    sub = df.copy()
    if col_mode in sub.columns:
        sub = sub[sub[col_mode].astype(str).str.strip() == str(mode)]
    sub = sub[sub[col_N].astype(float) == float(N)]
    if len(sub) == 0:
        raise ValueError(f"No row found for mode={mode}, N={N}")
    return sub.tail(1).iloc[0]  # if duplicates exist, use last appended

def fmt(x): 
    return f"{x:.6g}"

# Grab runs
analog_100k = pick_run(df, -1, 100_000)
vr_100k     = pick_run(df,  0, 100_000)
analog_10M  = pick_run(df, -1, 10_000_000)

# (a) Fair comparison: same N
R_ratio_a = float(analog_100k[col_R]) / float(vr_100k[col_R])
T_ratio_a = float(vr_100k[col_T]) / float(analog_100k[col_T])
F_ratio_a = float(vr_100k[col_F]) / float(analog_100k[col_F])

print("\n(a) Fair comparison (N = 100k): Analog vs VR")
print(f"  R_analog / R_VR        = {fmt(R_ratio_a)}")
print(f"  T_VR / T_analog        = {fmt(T_ratio_a)}")
print(f"  FOM_VR / FOM_analog    = {fmt(F_ratio_a)}")

# (b) Dramatic comparison: 10M vs 100k
events_ratio_b = float(analog_10M[col_N]) / float(vr_100k[col_N])
R_ratio_b      = float(analog_10M[col_R]) / float(vr_100k[col_R])
T_ratio_b      = float(analog_10M[col_T]) / float(vr_100k[col_T])
F_ratio_b      = float(vr_100k[col_F]) / float(analog_10M[col_F])

print("\n(b) VR comparison: Analog 10M vs VR 100k")
print(f"  N_10M / N_100k         = {fmt(events_ratio_b)}")
print(f"  R_10M / R_VR100k       = {fmt(R_ratio_b)}")
print(f"  T_10M / T_VR100k       = {fmt(T_ratio_b)}")
print(f"  FOM_VR100k / FOM_10M   = {fmt(F_ratio_b)}")

## Final conclusion

This is the final synthesis of the hands-on. Use the numerical results you obtained above.
Do **not** answer with vague statements.

---

### (a) Comparison - Analog 100k vs VR 100k 

1. **Relative uncertainty:** did it improve? By what factor?  
   → $R_{\text{analog}} / R_{\text{VR}} =$ ________

2. **Runtime:** did VR cost extra time? By what factor?  
   → $T_{\text{VR}} / T_{\text{analog}} =$ ________

3. **Efficiency:** did the FOM improve? By what factor?  
   → $\mathrm{FOM}_{\text{VR}} / \mathrm{FOM}_{\text{analog}} =$ ________

**One-sentence quantitative takeaway:**  

Write **one sentence** that includes at least **one numerical factor**

(e.g. “VR reduced the relative error by a factor X at comparable cost”).

---

### (b) Comparison - Analog 10M vs VR 100k

1. **Events:** how many fewer events did VR use?  
   → $N_{10M} / N_{100k} =$ ________×

2. **Precision:** how does relative uncertainty compare?  
   → $R_{10M} / R_{\text{VR}} =$ ________×

3. **Efficiency:** compare FOMs  
   → $\mathrm{FOM}_{\text{VR}} / \mathrm{FOM}_{10M} =$ ________×

**One-sentence quantitative takeaway:**  

Write **one sentence** that includes at least **one numerical factor**

(e.g. “VR reduced the relative error by a factor X at comparable cost”).

---

### Link back to the three checks (be specific)

- **Accuracy:**  
  State whether the VR and analog results are statistically compatible,
  and mention the criterion you used (e.g. z-score, overlap of uncertainties).

- **Efficiency:**  
  State by how much the FOM improved and what dominated the gain
  (variance reduction vs runtime increase).

- **Stability (weights / behaviour):**  
  State whether the VR estimator showed smooth behaviour or signs of instability,
  and point to at least one observable (e.g. Av.Tr.WGT trend, absence of spikes).